In [429]:
"""
Истпытательный скрипт по Учебным дисциплинам
"""

'\nИстпытательный скрипт по Учебным дисциплинам\n'

In [430]:
import pandas as pd
import numpy as np
import openpyxl
from docxtpl import DocxTemplate
import string
import time
import re

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.expand_frame_repr', False)  # Не переносить строки
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=FutureWarning, module='openpyxl')

In [431]:
df_plan_ud = pd.read_excel('data/Автозаполнение РП.xlsx',sheet_name='План УД',usecols='A:F')
df_plan_ud.dropna(inplace=True, thresh=1)  # удаляем пустые строки

In [432]:
df_plan_ud.columns = ['Раздел','Тема','Количество_часов','Практика','Вид_занятия','СРС']

In [433]:
df_plan_ud

,Раздел,Тема,Количество_часов,Практика,Вид_занятия,СРС
0,1 курс \n1 семестр,NaN,NaN,NaN,NaN,NaN
1,NaN,Грамматически основы построения предложений,2.0,NaN,урок,NaN
2,NaN,История железнодорожного транспорта.,2.0,NaN,практическое занятие,NaN
3,NaN,История железнодорожного транспорта.,2.0,NaN,практическое занятие,NaN
4,NaN,Первая железная дорога в России.,2.0,NaN,практическое занятие,NaN
...,...,...,...,...,...,...
64,NaN,Правила перевозки домашних животных,2.0,NaN,практическое занятие,NaN
65,NaN,Правила перевозки домашних животных,2.0,NaN,практическое занятие,NaN
66,NaN,"Вещества и предметы, запрещенные к провозу на ...",2.0,NaN,практическое занятие,NaN
67,NaN,Итоговая контрольная работа,2.0,NaN,практическое занятие,NaN


In [434]:
df_plan_ud['Раздел'].fillna('Пусто',inplace=True)

In [435]:
borders=df_plan_ud[df_plan_ud['Раздел'].str.contains('семестр')].index # получаем индексы строк где есть слово семестр

In [436]:
borders

Int64Index([0, 18, 41, 56], dtype='int64')

In [437]:
part_df = [] # список для хранения кусков датафрейма
previos_border = -1
# делим датафрем по границам
for value_border in borders:
    part = df_plan_ud.iloc[previos_border:value_border]
    part_df.append(part)
    previos_border = value_border


In [438]:
# добавляем последнюю часть
last_part = df_plan_ud.iloc[borders[-1]:]
part_df.append(last_part)

In [439]:
part_df.pop(0) # удаляем нулевой элемент так как он пустой

,Раздел,Тема,Количество_часов,Практика,Вид_занятия,СРС


In [440]:
main_df = pd.DataFrame(columns=['Раздел','Тема','Количество_часов','Практика','Вид_занятия','СРС']) #  создаем базовый датафрейм

In [441]:
main_df

,Раздел,Тема,Количество_часов,Практика,Вид_занятия,СРС


In [442]:
part_df[1]

,Раздел,Тема,Количество_часов,Практика,Вид_занятия,СРС
18,1 курс \n2 семестр,NaN,NaN,NaN,NaN,NaN
19,Пусто,Речевые клише и правила построения фраз,2.0,NaN,урок,NaN
20,Пусто,Работа и обязанности проводника,2.0,NaN,практическое занятие,NaN
21,Пусто,Работа и обязанности проводника,2.0,NaN,практическое занятие,NaN
22,Пусто,Грамматические основы построения диалогов,2.0,NaN,урок,NaN
23,Пусто,"Помощь пассажиру в поиске нужной платформы, н...",2.0,NaN,практическое занятие,NaN
24,Пусто,"Помощь пассажиру в поиске нужной платформы, н...",2.0,NaN,практическое занятие,NaN
25,Пусто,Посадка в вагон. Распределение багажа.,2.0,NaN,практическое занятие,NaN
26,Пусто,Посадка в вагон. Распределение багажа,2.0,NaN,практическое занятие,NaN
27,Пусто,Информирование пассажиров перед отправлением п...,2.0,NaN,практическое занятие,NaN


In [443]:
lst_type_lesson= ['урок','практическое занятие','лабораторное занятие','курсовая работа (КП)'] # список типов занятий
for df in part_df[:2]:
    dct_sum_result = {key:0 for key in lst_type_lesson} # создаем словарь для подсчета значений
    for type_lesson in lst_type_lesson:
        _df = df[df['Вид_занятия'] == type_lesson] # фильтруем датафрейм
        _df['Количество_часов'].fillna(0,inplace=True)
        _df['Количество_часов'] = _df['Количество_часов'].astype(int)
        dct_sum_result[type_lesson] = _df['Количество_часов'].sum()
        # создаем строку с описанием
        margint_text = 'Итого часов за семестр:\nиз них\nтеория\nпрактические занятия\nлабораторные занятия\nкурсовая работа (КП)'

        all_hours = sum(dct_sum_result.values()) # общая сумма часов

        theory_hours = dct_sum_result['урок'] # часы теории
        praktice_hours = dct_sum_result['практическое занятие'] # часы практики
        lab_hours = dct_sum_result['лабораторное занятие'] # часы лабораторных
        kurs_hours = dct_sum_result['курсовая работа (КП)'] # часы курсовых

        value_text = f'{all_hours}\n""\n{theory_hours}\n{praktice_hours}\n{lab_hours}\n{kurs_hours}' # строка со значениями
        itog_row = {'Тема':margint_text,'Количество_часов':value_text} # создаем строку

    df.loc[len(df.index)] = itog_row # добавляем строку
    main_df = pd.concat([main_df,df],ignore_index=True)
        
        
        


In [444]:
main_df

,Раздел,Тема,Количество_часов,Практика,Вид_занятия,СРС
0,1 курс \n1 семестр,NaN,NaN,NaN,NaN,NaN
1,Пусто,Грамматически основы построения предложений,2.0,NaN,урок,NaN
2,Пусто,История железнодорожного транспорта.,2.0,NaN,практическое занятие,NaN
3,Пусто,История железнодорожного транспорта.,2.0,NaN,практическое занятие,NaN
4,Пусто,Первая железная дорога в России.,2.0,NaN,практическое занятие,NaN
5,Пусто,Первая железная дорога в России.,2.0,NaN,практическое занятие,NaN
6,Пусто,Современные железные дороги.,2.0,NaN,практическое занятие,NaN
7,Пусто,Современные железные дороги.,2.0,NaN,практическое занятие,NaN
8,Пусто,Контрольная работа.,2.0,NaN,урок,NaN
9,Пусто,Основные термины и определения. Грамматические...,2.0,NaN,практическое занятие,NaN


In [445]:
main_df.to_excel('fd.xlsx',index=False)

In [446]:

dct_sum_result = {key:0 for key in lst_type_lesson}

In [447]:
temp_df = part_df[1]


In [448]:
for type_lesson in lst_type_lesson:
    _df = temp_df[temp_df['Вид_занятия'] == type_lesson] # фильтруем датафрейм
    _df['Количество_часов'].fillna(0,inplace=True)
    _df['Количество_часов'] = _df['Количество_часов'].astype(int)
    dct_sum_result[type_lesson] = _df['Количество_часов'].sum()

In [449]:
dct_sum_result

{'урок': 4,
 'практическое занятие': 38,
 'лабораторное занятие': 0,
 'курсовая работа (КП)': 0}

In [450]:
# создаем строку с описанием
margint_text = 'Итого часов за семестр:\nиз них\nтеория\nпрактические занятия\nлабораторные занятия\nкурсовая работа (КП)'

In [451]:
all_hours = sum(dct_sum_result.values()) # общая сумма часов

In [452]:
theory_hours = dct_sum_result['урок'] # часы теории
praktice_hours = dct_sum_result['практическое занятие'] # часы практики
lab_hours = dct_sum_result['лабораторное занятие'] # часы лабораторных
kurs_hours = dct_sum_result['курсовая работа (КП)'] # часы курсовых

In [453]:
value_text = f'{all_hours}\n""\n{theory_hours}\n{praktice_hours}\n{lab_hours}\n{kurs_hours}' # строка со значениями

In [454]:
value_text

'42\n""\n4\n38\n0\n0'

In [455]:
temp_df.columns

Index(['Раздел', 'Тема', 'Количество_часов', 'Практика', 'Вид_занятия', 'СРС'], dtype='object')

In [456]:
itog_row = {'Тема':margint_text,'Количество_часов':value_text}

In [457]:
temp_df.loc[len(df.index)] = itog_row

In [458]:
temp_df

,Раздел,Тема,Количество_часов,Практика,Вид_занятия,СРС
18,1 курс \n2 семестр,NaN,NaN,NaN,NaN,NaN
19,Пусто,Речевые клише и правила построения фраз,2.0,NaN,урок,NaN
20,Пусто,Работа и обязанности проводника,2.0,NaN,практическое занятие,NaN
21,Пусто,Работа и обязанности проводника,2.0,NaN,практическое занятие,NaN
22,Пусто,Грамматические основы построения диалогов,2.0,NaN,урок,NaN
23,NaN,Итого часов за семестр:\nиз них\nтеория\nпракт...,"42\n""""\n4\n38\n0\n0",NaN,NaN,NaN
24,Пусто,"Помощь пассажиру в поиске нужной платформы, н...",2.0,NaN,практическое занятие,NaN
25,Пусто,Посадка в вагон. Распределение багажа.,2.0,NaN,практическое занятие,NaN
26,Пусто,Посадка в вагон. Распределение багажа,2.0,NaN,практическое занятие,NaN
27,Пусто,Информирование пассажиров перед отправлением п...,2.0,NaN,практическое занятие,NaN
